In [29]:
import numpy as np

from numpy.testing import assert_allclose


### Часть I. Постройте отражение Хаусхолдера для вектора.

Дан вектор $\mathbf{x}$ и плоскость, заданная вектором нормали $\mathbf{u}$. Преобразование Хаусхолдера отражает $\mathbf{x}$ относительно плоскости.

Матрица преобразований Хаусхолдера:
$$ \mathbf{H} = \mathbf{1} - 2 \mathbf{u} \mathbf{u}^T $$

Если даны два вектора $\mathbf{x}$ и $\mathbf{y}$ одинаковой длины, поворот, преобразующий $\mathbf{x}$ в $\mathbf{y}$ называется преобразованием Хаусхолдера с
$$ \mathbf{u} = \frac{\mathbf{x} - \mathbf{y}}{\left|\mathbf{x} - \mathbf{y}\right|} $$

Напишите функцию, преобразующую заданный вектор $\mathbf{x} = (x_1, \dots, x_n)$ в $\mathbf{y} = (\left|\mathbf{x}\right|, 0, \dots, 0)^T$, используя преобразование Хаусхолдера.


In [30]:
def householder(vec):
    """Создает отражение Хаусхолдера, преобразующее 2-ю, 3-ю и т.д. компоненты вектора vec в нули.
    
    Parameters
    ----------
    vec : array-like of floats, shape (n,)
        Введенный вектор
    
    Returns
    -------
    outvec : array of floats, shape (n,)
        Преобразованный вектор, причем ``outvec[1:]==0`` и ``|outvec| == |vec|``
    H : array of floats, shape (n, n)
        Ортогональная матрица отражений Хаусхолдера
    """
    vec = np.asarray(vec, dtype=float)
    if vec.ndim != 1:
        raise ValueError("vec.ndim = %s, expected 1" % vec.ndim)
    
    ###BEGIN SOLUTION
    module = np.linalg.norm(vec)
    y = np.zeros_like(vec)
    y[0] = module
    u = np.zeros_like(vec)
    u = (vec-y)/np.linalg.norm(vec-y)
    if abs(u[0])<1e-6:
        u[0] = -(sum(vec**2)-vec[0]**2)/(vec[0]+module) 
    U = u.T
    H = np.identity(np.shape(vec)[0]) - 2 * np.dot(u.reshape(-1,1), U.reshape(1,-1))
    
        
    return y, H
    ###END SOLUTION

Протестируйте свою функцию на следующих примерах:

In [31]:
# Тест I.1 (10% итоговой оценки).

v = np.array([1, 2, 3])
v1, h = householder(v)

assert np.allclose(np.dot(h, v1), v)
assert np.allclose(np.dot(h, v), v1)

In [32]:
# Test I.2 (10% итоговой оценки).

rndm = np.random.RandomState(1234)

vec = rndm.uniform(size=7)
v1, h = householder(vec)

assert np.allclose(np.dot(h, v1), vec)


### Part II. Вычисление $\mathrm{QR}$ - разложения матрицы.

Дана прямоугольная $m\times n$ матрица $\mathbf{A}$. Выполните отражение Хаусхолдера матрицы $\mathbf{H}_1$, преобразующее первый столбец матрицы $\mathbf{A}$ (назовем результат $\mathbf{A}^{(1)}$)
$$ \mathbf{H}_1 \mathbf{A} =% \begin{pmatrix} \times &amp; \times &amp; \times &amp; \dots &amp; \times \\ 0 &amp; \times &amp; \times &amp; \dots &amp; \times \\ 0 &amp; \times &amp; \times &amp; \dots &amp; \times \\ &amp;&amp; \dots&amp;&amp; \\ 0 &amp; \times &amp; \times &amp; \dots &amp; \times \\ \end{pmatrix}% \equiv \mathbf{A}^{(1)}\;. $$

Теперь рассмотрим нижнюю правую часть матрицы $\mathbf{A}^{(1)}$ и выполним преобразование Хаусхолдера, действующее на 2 столбец $\mathbf{A}$:
$$ \mathbf{H}_2 \mathbf{A}^{(1)} =% \begin{pmatrix} \times &amp; \times &amp; \times &amp; \dots &amp; \times \\ 0 &amp; \times &amp; \times &amp; \dots &amp; \times \\ 0 &amp; 0 &amp; \times &amp; \dots &amp; \times \\ &amp;&amp; \dots&amp;&amp; \\ 0 &amp; 0 &amp; \times &amp; \dots &amp; \times \\ \end{pmatrix}% \equiv \mathbf{A}^{(2)} \;. $$

Повторяя процесс $n-1$ раз получим
$$ \mathbf{H}_{n-1} \cdots \mathbf{H}_2 \mathbf{H}_1 \mathbf{A} = \mathbf{R} \;, $$

где $\mathbf{R}$ верхнетреугольная матрица. Так как каждая $\mathbf{H}_k$ ортогональна, таковым будет и их произведение. Обратная от ортогональной также есть матрица ортогональная. Таким образом, алгоритм создает $\mathrm{QR}$ - разложение матрицы $\mathbf{A}$.

Напишите функцию, получающую прямоугольную матрицу $A$ и возвращающую Q и R - компоненты $QR$ - разложения $A$.


In [14]:
def qr_decomp(a):
    """Вычисляет QR - разложение матрицы.
    
    Parameters
    ----------
    a : ndarray, shape(m, n)
        Введенная матрица
    
    Returns
    -------
    q : ndarray, shape(m, m)
        Ортогональная матрица
    r : ndarray, shape(m, n)
        Верхнетреугольная матрица
        
    Example
    --------
    >>> a = np.random.random(size=(3, 5))
    >>> q, r = qr_decomp(a)
    >>> np.assert_allclose(np.dot(q, r), a)
    
    """
    a1 = np.array(a, copy=True, dtype=float)
    m, n = a1.shape
    
    ###BEGIN SOLUTION
    a1 = np.array(a, copy=True, dtype=float)
    m, n = a1.shape
    H = np.eye(m)
   
    for i in range(n if n<m else m-1):
        matrix = a1[i:, i:]
        vec = matrix[:,0]
        Hi = np.eye(m)
        Hi[i:, i:] = householder(vec)[1]
        a1 = Hi @ a1
        H = Hi @ H
    return H.T, H@a
    ###END SOLUTION

In [15]:
# можете запустить данную операцию для более красивого вывода: нули вместо `1e-16` и т.д.

np.set_printoptions(suppress=True)

In [19]:
# Тест II.1 (20% итоговой оценки)

rndm = np.random.RandomState(1234)
a = rndm.uniform(size=(5, 3))
q, r = qr_decomp(a)

# тестируем, что Q ортогональна
assert np.allclose(np.dot(q, q.T), np.eye(5), atol=1e-10)

# проверяем разложение
assert np.allclose(np.dot(q, r), a)


Теперь сравните ваше разложение с разложением, полученным библиотечной функцией (содержащей соответствующие функции библиотеки LAPACK)

In [22]:
from scipy.linalg import qr
qq, rr = qr(a)

assert np.allclose(np.dot(qq, rr), a)

Проверьте, согласуются ли ваши q и r с qq и rr. Объясните.
Оставьте пояснения здесь (10% итоговой оценки) 


### Часть III. Безматричная реализация.

Отметим необычную структуру матрицы Хаусхолдера: матрица поворота $\mathbf{H}$ полностью характеризуется вектором отражения $\mathbf{u}$. Заметим, также, что вычислительная сложность операции отражения матрицы сильно зависит от порядка операций:
$$ \left( \mathbf{u} \mathbf{u}^T \right) \mathbf{A} \qquad \textrm{is } O(m^2 n)\;, $$

тогда как $$ \mathbf{u} \left( \mathbf{u}^T \mathbf{A} \right) \qquad \textrm{is } O(mn) $$

Таким образом, следует избегать формирований матриц $\mathbf{H}$. Вместо этого можно сохранять сами векторы отражения $\mathbf{u}$ и производить умножение произвольной матрицы на $\mathbf{Q}^T$, например, как отдельную функцию (класс).

Напишите функцию, выполняющую QR - разложение матрицы без формирования матриц $\mathbf{H}$ и возвращающую матрицу $\mathbf{R}$, а также вектор отражения.

Напишите вторую функцию, которая использует вектор отражения чтобы перемножить матрицу с $\mathbf{Q}^T$. Убедитесь, что вы добавили достаточно комментариев, следующих вашим выкладкам. Напишите тесты.

(40% итоговой оценки)


In [28]:
def a_to_RU(a):
    a1 = a.copy()
    m,n = a.shape
    U = np.zeros_like(a)

    for i in range(n if n<m else m-1):#Проверяем, какое измерение матрицы больше
        vec = a1[i:,i]#Вектор, который хотим повернуть - от диагонали и ниже
        y = np.zeros_like(vec)
        y[0] = np.linalg.norm(vec)
        u = np.zeros(m-i)#Наш reflection vector(вектор отражения?)
        u = (vec-y)/np.linalg.norm(vec-y)#Заполняем как в Хаусхолдере
        if abs(u[0])<1e-6:
            u[0] = -(sum(vec**2)-vec[0]**2)/(vec[0]+y[0])#Чтобы избежать зануления в случае, когда векторы почти параллельны
        a1[i:,i:] = a1[i:,i:] -  2 * np.dot(u.reshape(-1,1), (u @ a1[i:,i:]).reshape(1,-1))#Преобразование из инструкции
        
        U[i:,i] = u#Нужный столбик матрицы заполняем нашим вектором
    return a1, U

def aU_to_R(a, U):
    a1 = a.copy()
    m,n = a.shape
    R = a1.copy()

    for i in range(n if n<m else m-1):
        u = U[i:,i]#Берем i-тый вектор отражения
        R[i:,i:] = R[i:,i:] - 2 * np.dot(u.reshape(-1,1), (u @ R[i:,i:]).reshape(1,-1))#Находим (1-2u_i@u_i.T)a 
    return R
#Тесты 
print("Test 1: vertical rectangular matrix")
a = rndm.uniform(size=(5,3))
R, U = a_to_RU(a)
print('a',a,'U', U,'R from the 1 function',R,'R from the 2 function', aU_to_R(a, U),
      'R from library',qr(a)[1],sep='\n')

print("Test 2: square matrix")
a = rndm.uniform(size=(5,5))
R, U = a_to_RU(a)
print('a',a,'U', U,'R from the 1 function',R,'R from the 2 function', aU_to_R(a, U),
      'R from library',qr(a)[1],sep='\n')

print("Test 3: horizontal rectangular matrix")
a = rndm.uniform(size=(3,5))
R, U = a_to_RU(a)
print('a',a,'U', U,'R from the 1 function',R,'R from the 2 function', aU_to_R(a, U),
      'R from library',qr(a)[1],sep='\n')

print("Test 4: bad matrix")
a = rndm.uniform(size=(5,3))
a[0,0] = 100
R, U = a_to_RU(a)
print('a',a,'U', U,'R from the 1 function',R,'R from the 2 function', aU_to_R(a, U),
      'R from library',qr(a)[1],sep='\n')

#И снова все совпадает с точностью до знака
#А значит, разложение верное

Test 1: vertical rectangular matrix
a
[[0.56119619 0.50308317 0.01376845]
 [0.77282662 0.88264119 0.36488598]
 [0.61539618 0.07538124 0.36882401]
 [0.9331401  0.65137814 0.39720258]
 [0.78873014 0.31683612 0.56809865]]
U
[[-0.57604006  0.          0.        ]
 [ 0.40205315 -0.30471224  0.        ]
 [ 0.32015198 -0.79045901 -0.78244581]
 [ 0.48545419  0.38872927 -0.62222608]
 [ 0.41032676 -0.3622355   0.0247639 ]]
R from the 1 function
[[ 1.66846046  1.11993758  0.80038785]
 [ 0.          0.55520169 -0.18167936]
 [ 0.         -0.          0.27611658]
 [ 0.          0.         -0.        ]
 [ 0.         -0.          0.        ]]
R from the 2 function
[[ 1.66846046  1.11993758  0.80038785]
 [ 0.          0.55520169 -0.18167936]
 [ 0.         -0.          0.27611658]
 [ 0.          0.         -0.        ]
 [ 0.         -0.          0.        ]]
R from library
[[-1.66846046 -1.11993758 -0.80038785]
 [ 0.         -0.55520169  0.18167936]
 [ 0.          0.         -0.27611658]
 [ 0.          